In [ ]:
import pandas as pd
import scipy.sparse
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the job offers and job applications datasets
job_offers = pd.read_csv('/content/drive/MyDrive/JobOffersSample.csv')
job_seekers= pd.read_csv('/content/drive/MyDrive/CvDatasetFinal (3).csv')

In [ ]:
job_offers.head()

,Unnamed: 0,Job post,Company name,Job description,Required skills,Location,Company rating,Company review,Experience required
0,3391,softwar engin,ibm,softwar develop posit ibm enterpris data prote...,linux project manag schedul troubleshoot forec...,pune,4.2,13315.0,5-8 yr
1,18688,analyst-ord cash process,accentur,skill requir order cash process bill process d...,order manag bill consult order process oper jo...,noida,4.1,26567.0,3-5 yr
2,195,senior site reliabl engin s/4hana cloud,sap,bachelor master degre comput scienc engin busi...,s/4hana cloud skill cloud sap comput scienc au...,bangalore/bengaluru,4.4,1116.0,10 yr
3,28603,sap co manag account softwar develop engin,accentur,educ qualif 16 yr regular educ bachelor degre ...,sap co project manag consult sap p commun s4 h...,hyderabad/secunderabad,4.1,26528.0,4-6 yr
4,2252,softwar qualiti assur leader aspic,zf india privat limit,role ensur execut qualiti assur well sign-off ...,skill qualiti assur autom product engin analyt...,chennai,3.8,424.0,10-15 yr


In [ ]:
job_seekers.head()


,Category,Name,Email,Phone,Education,Skills,Experience,Experience_Rating
0,Data Science,Lauren Armstrong,lauren.armstrong@gmail.com,372.165.3732x4577,detail data science assurance associate data s...,skill programming language python panda numpy ...,24 month jquery 24 month python 24 monthscompa...,5
1,Data Science,Kristen Lambert,kristen.lambert@gmail.com,410.381.6487x49465,detail may 2013 may 2017 uitrgpv data scientis...,detail python,le 1 year month statsmodels 12 month aws le 1 ...,5
2,Data Science,Jason Montes,jason.montes@gmail.com,(962)653-8542x1217,detail january 2019 btech electrical electroni...,skill programming matlab python java labview p...,le 1 year month excel le 1 year month machine ...,5
3,Data Science,Molly Jackson,molly.jackson@gmail.com,110.048.2076x99778,detail january 2017 january 2018 pgdm business...,skill python sap hana tableau sap hana sql sap...,6 month retail 6 month retail marketing 6 mont...,5
4,Data Science,Michele Smith,michele.smith@gmail.com,(759)023-5239x309,detail mca ymcaust faridabad haryana data scie...,detail data structure,le 1 year month c le 1 year month data analysi...,5


In [ ]:
# Check for missing values in the job offers dataframe
print("Missing values in job_offers:")
print(job_offers.isna().sum())

# Check for missing values in the job applications dataframe
print("Missing values in job_applications:")
print(job_seekers.isna().sum())

Missing values in job_offers:
Unnamed: 0             0
Job post               0
Company name           0
Job description        0
Required skills        0
Location               0
Company rating         0
Company review         0
Experience required    0
dtype: int64
Missing values in job_applications:
Category             0
Name                 0
Email                0
Phone                0
Education            0
Skills               0
Experience           0
Experience_Rating    0
dtype: int64


In [ ]:
# Drop rows with missing values in job_offers and job_seekers dataframes
job_offers.dropna(inplace=True)
job_seekers.dropna(inplace=True)

In [ ]:
# Preprocess job seekers data
tfidf = TfidfVectorizer(stop_words='english')
job_seekers_matrix = tfidf.fit_transform(job_seekers['Skills'])

In [ ]:
import scipy.sparse
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def hybrid_recommendations(job_offer, job_offers, job_seekers, top_n):

    # Transform the job offer string into a count matrix
    cv = CountVectorizer(vocabulary=tfidf.vocabulary_)
    job_offer_matrix = cv.transform([job_offer])

    # Preprocess job seekers data
    job_seekers_tfidf = TfidfVectorizer(stop_words='english')
    job_seekers_matrix = job_seekers_tfidf.fit_transform(job_seekers['Skills'])

    # Calculate the cosine similarity between the job offer and job seekers
    cosine_sim = cosine_similarity(job_offer_matrix, job_seekers_matrix)

    # Get the indices of the top_n job seekers
    seeker_indices = cosine_sim[0].argsort()[-top_n:][::-1]

    # Get the corresponding names, skills, and resumes of the top_n job seekers
    names = job_seekers.iloc[seeker_indices]['Name'].values
    skills = job_seekers.iloc[seeker_indices]['Skills'].values


    return names, skills, cosine_sim[0][seeker_indices]


In [ ]:
import pandas as pd

# Create a TfidfVectorizer object for job offers
job_offers_tfidf = TfidfVectorizer(stop_words='english')
job_offers_matrix = job_offers_tfidf.fit_transform(job_offers['Job description'])

# Define the number of top matches to retrieve
top_n = 5

# Iterate over each job offer and display the top n matching job seekers
for i, job_offer in job_offers.iterrows():
    names, skills, cosine_sims = hybrid_recommendations(job_offer['Job description'], job_offers_matrix, job_seekers, top_n)
    print(f"Top {top_n} matches for Job Offer {i+1}:")
    for j, name in enumerate(names):
        print(f"{j+1}. Name: {name}, Skills: {skills[j]}, Cosine Similarity: {cosine_sims[j]}")
    print()


Streaming output truncated to the last 5000 lines.
5. Name: Patricia Harper, Skills: set talend big data informatica power center microsoft sql server sql platform 621 management studio workbench aws service talend administration console microsoft visual redshift tac studio athena data warehouse concept star sql s3 schema fact dimension data modeling data integration microsoft access detail january 2012 january 2016 mumbai maharashtra university mumbai january 2012 cbse technology kochi kerala st francis talend etl developer talend etl developer tata consultancy service detail data warehouse skill profound insight determine priority schedule work meet critical deadline company tata consultancy service description prepared etl mapping document every mapping data migration document smooth transfer project development testing environment production environment performed unit testing system testing validate data load target troubleshoot long running job fixed issue, Cosine Similarity: 0.29

In [ ]:
job_offer = "Data Scientist with experience in machine learning and Python"
top_n = 5

names, skills, cosine_sims = hybrid_recommendations(job_offer, job_offers, job_seekers, top_n)

for name, skill, cosine_sim in zip(names, skills, cosine_sims):
    print('Name:', name)
    print('Skills:', skill)
    print('Cosine similarity:', cosine_sim)
    print('---------------------')

Name: Stacy Anderson
Skills: skill python tableau data visualization studio machine learning statistic iabac certified data scientist versatile detail machine learning
Cosine similarity: 0.6596029023562244
---------------------
Name: Barbara Shelton
Skills: skill python tableau data visualization studio machine learning statistic iabac certified data scientist versatile detail machine learning
Cosine similarity: 0.6596029023562244
---------------------
Name: Christine Myers
Skills: skill python tableau data visualization studio machine learning statistic iabac certified data scientist versatile detail machine learning
Cosine similarity: 0.6596029023562244
---------------------
Name: Joseph Burns
Skills: skill python tableau data visualization studio machine learning statistic iabac certified data scientist versatile detail machine learning
Cosine similarity: 0.6596029023562244
---------------------
Name: Autumn Burgess
Skills: detail python
Cosine similarity: 0.4472135954999579
-------